In [1]:
import os
import pandas as pd
from pandas import Series, DataFrame

Data source:

- https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/9FEJ7C

In [2]:
## Get current working directory
print("Current working directory:", os.getcwd())

## Find path to raw data (for the raw files)
raw_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'raw_data'))

## Find path to folder for saving cleaned csv
cleaned_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'cleaned_data'))


## Find path to folder for inflation
inflation_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'inflation'))

Current working directory: /Users/jennagreene/Documents/GitHub/HATCH_data/reading_files


In [3]:

target_inflation_file = "/A001RG3A086NBEA.xls"
target_inflation_filepath = inflation_data_path + target_inflation_file

target_file = '/cost data/LiIonDataSeries_represonly_withcover.xlsx'
target_filepath = raw_data_path + target_file

In [4]:
## adjusting for inflation from 2018 USD to 2022 USD
nipa = pd.read_excel(target_inflation_filepath, header=10)

year_list = []
for x in nipa['observation_date']:
    x = int(str(x)[:4])
    year_list.append(x)
nipa['Year'] = year_list
nipa.set_index('Year', drop=True, inplace=True)
nipa.drop(columns='observation_date', inplace=True)
nipa = nipa.transpose()

infl_2018_2022 = float(nipa[2022]/nipa[2018])
infl_2018_2022

/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_31508/438783119.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  infl_2018_2022 = float(nipa[2022]/nipa[2018])


1.1528881798983042

In [5]:
li_ion = pd.read_excel(target_filepath, 
                       sheet_name='RepreSeries_Price_All_Cells', usecols=[0,1], index_col=0)
li_ion = li_ion.transpose()
li_ion.iloc[0] = li_ion.iloc[0] * infl_2018_2022
li_ion['Unit'] = '2022 USD/kWh'
li_ion['Data Source'] = 'Ziegler & Trancik'
li_ion['Spatial Scale'] = 'Global'
li_ion['Country Code'] = 'World'
li_ion['Country Name'] = 'World'
li_ion['Metric'] = 'Price'
li_ion['Technology Name'] = 'Lithium-Ion Cells'
li_ion['ID'] = li_ion['Technology Name'] + '_' + li_ion['Metric'] + '_' + li_ion['Country Code']
li_ion.set_index('ID', drop=True, inplace=True)
li_ion.columns.name = None
li_ion

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2016,2017,2018,Unit,Data Source,Spatial Scale,Country Code,Country Name,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Lithium-Ion Cells_Price_World,8673.505347,6957.594686,5539.021034,6323.165382,5934.641484,4998.520505,4280.416059,3443.759846,2608.458359,2533.83099,...,280.768969,219.455051,209.486705,2022 USD/kWh,Ziegler & Trancik,Global,World,World,Price,Lithium-Ion Cells


In [6]:
output_file = 'li_ion_prices.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)

li_ion.to_csv(output_file_path)
print("Data saved to:", output_file_path)

Data saved to: /Users/jennagreene/Documents/GitHub/HATCH_data/cleaned_data/li_ion_prices.csv
